In [1]:
import copy
import os.path as osp
from glob import glob
from tqdm import tqdm

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config
print(get_compiling_cuda_version())
print(get_compiler_version())

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import train_detector, set_random_seed, init_detector, inference_detector

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from IPython.core.debugger import set_trace

1.7.0+cu101 True
2.7.0
10.1
GCC 7.3


# Utils

In [2]:
def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
            return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

def precision_calc(gt_boxes, pred_boxes):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            dist = abs(box1[0]-box2[0])
            if dist > 4:
                continue
            
            iou_score = iou(box1[1:], box2[1:])
            
            if iou_score < 0.35:
                continue
            else:
                cost_matix[i,j]=0
    
    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp=0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i,j]==0:
            tp+=1
        else:
            fp+=1
            fn+=1
    return tp, fp, fn

# Load GT

In [3]:
video_labels = pd.read_csv('/home/thinh/nfl/train_labels.csv')
video_labels = video_labels[video_labels['frame'] != 0].reset_index(drop=True)

In [4]:
video_labels.head()

,gameKey,playID,view,video,frame,label,left,width,top,height,impact,impactType,confidence,visibility
0,57583,82,Endzone,57583_000082_Endzone.mp4,1,H30,629,19,40,24,NaN,NaN,NaN,NaN
1,57583,82,Endzone,57583_000082_Endzone.mp4,1,V72,443,22,344,16,NaN,NaN,NaN,NaN
2,57583,82,Endzone,57583_000082_Endzone.mp4,1,V86,871,21,359,17,NaN,NaN,NaN,NaN
3,57583,82,Endzone,57583_000082_Endzone.mp4,1,V74,771,19,345,15,NaN,NaN,NaN,NaN
4,57583,82,Endzone,57583_000082_Endzone.mp4,1,V34,549,26,461,20,NaN,NaN,NaN,NaN


# Config

In [5]:
# Specify the path to model config and checkpoint file
valid_epoch = 28
config_file = './cascade_rcnn_r50_fpn.py'
checkpoint_file = f'/home/thinh/nfl/mmdet-models/model_a1_44/epoch_{valid_epoch}.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

/home/thinh/nfl/mmdetection/mmdet/apis/inference.py:46: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


# Predict

In [6]:
test_lists = []

with open('video_valid_44.txt') as file:
    test_video_names = file.readline()
    for video_name in test_video_names[2:-2].split("', '"):
        test_lists.append(f'{video_name}_Endzone.mp4')
        test_lists.append(f'{video_name}_Sideline.mp4')

In [7]:
test_lists

['57583_000082_Endzone.mp4',
 '57583_000082_Sideline.mp4',
 '57586_004152_Endzone.mp4',
 '57586_004152_Sideline.mp4',
 '57911_000147_Endzone.mp4',
 '57911_000147_Sideline.mp4',
 '57997_003691_Endzone.mp4',
 '57997_003691_Sideline.mp4',
 '57680_002206_Endzone.mp4',
 '57680_002206_Sideline.mp4',
 '58095_004022_Endzone.mp4',
 '58095_004022_Sideline.mp4',
 '57906_000718_Endzone.mp4',
 '57906_000718_Sideline.mp4',
 '58005_001254_Endzone.mp4',
 '58005_001254_Sideline.mp4',
 '57679_003316_Endzone.mp4',
 '57679_003316_Sideline.mp4',
 '58103_003494_Endzone.mp4',
 '58103_003494_Sideline.mp4',
 '57998_002181_Endzone.mp4',
 '57998_002181_Sideline.mp4',
 '58048_000086_Endzone.mp4',
 '58048_000086_Sideline.mp4']

In [8]:
gt_df = video_labels[video_labels['video'].isin(test_lists) & video_labels['impact']==1]

In [9]:
gameKey = []
playID = []
view = []
video_name = []
frame_num = []
left = []
width = []
top = []
height = []
score = []

for test_list in tqdm(test_lists):
    file_name = f'/home/thinh/nfl/train/{test_list}'
    video = mmcv.VideoReader(file_name)
    
    for i, frame in enumerate(video):
        result = inference_detector(model, frame)
        
        for j in range(len(result[0])):
#             score = result[0][j][4]
#             if score > 0.9:
            gameKey.append(int(file_name.split('_')[0].split('/')[-1]))
            playID.append(int(file_name.split('_')[1]))
            view.append(file_name.split('_')[2][:-4])
            video_name.append(file_name.split('/')[-1])
            frame_num.append(i+1)
            
            score.append(result[0][j][4])
            left.append(int(result[0][j][0]))
            top.append(int(result[0][j][1]))
            width.append(int(abs(result[0][j][2] - result[0][j][0])))
            height.append(int(abs(result[0][j][3] - result[0][j][1])))

100%|██████████| 24/24 [41:26<00:00, 103.59s/it]


In [10]:
test_df = pd.DataFrame({'gameKey': gameKey,
             'playID': playID,
             'view': view,
             'video': video_name,
             'frame': frame_num,
             'left': left,
             'width': width,
             'top': top,
             'height': height,
             'score': score}
             )

In [11]:
test_df.to_csv(f'/home/thinh/nfl/mmdet-models/model_a1_44/epoch_{valid_epoch}.csv', index=False)
test_df.head()

,gameKey,playID,view,video,frame,left,width,top,height,score
0,57583,82,Endzone,57583_000082_Endzone.mp4,1,556,22,326,18,0.536193
1,57583,82,Endzone,57583_000082_Endzone.mp4,2,556,22,326,18,0.507391
2,57583,82,Endzone,57583_000082_Endzone.mp4,3,556,21,326,18,0.483377
3,57583,82,Endzone,57583_000082_Endzone.mp4,4,555,22,326,18,0.477639
4,57583,82,Endzone,57583_000082_Endzone.mp4,9,554,22,328,17,0.428056


In [12]:
gt_df['bot'] = gt_df['top'] + gt_df['height']
gt_df['right'] = gt_df['left'] + gt_df['width']
test_df['bot'] = test_df['top'] + test_df['height']
test_df['right'] = test_df['left'] + test_df['width']

/home/thinh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
test_df.shape

(7401, 12)

In [14]:
gt_df.shape

(449, 16)

In [15]:
ftp, ffp, ffn = [], [], []
for count, video in enumerate(set(gt_df['video'])):
    pred_boxes = test_df[test_df['video']==video][["frame",'left', 'top', 'right', 'bot']].to_numpy()
    gt_boxes = gt_df[gt_df['video']==video][["frame",'left', 'top', 'right', 'bot']].to_numpy()
    tp, fp, fn = precision_calc(gt_boxes, pred_boxes)
    ftp.append(tp)
    ffp.append(fp)
    ffn.append(fn)

tp = np.sum(ftp)
fp = np.sum(ffp)
fn = np.sum(ffn)
precision = tp / (tp + fp + 1e-6)
recall =  tp / (tp + fn +1e-6)
f1_score = 2*(precision*recall)/(precision+recall+1e-6)
print(f'TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {precision:.4f}, RECALL: {recall:.4f}, F1 SCORE: {f1_score:.4f}')

TP: 229, FP: 7172, FN: 220, PRECISION: 0.0309, RECALL: 0.5100, F1 SCORE: 0.0583


In [16]:
# Epoch 1: 1689 - 449 - TP: 104, FP: 1585, FN: 345, PRECISION: 0.0616, RECALL: 0.2316, F1 SCORE: 0.0973
# Epoch 2: 5950 - 449 - TP: 194, FP: 5756, FN: 255, PRECISION: 0.0326, RECALL: 0.4321, F1 SCORE: 0.0606
# Epoch 3: 11023 - TP: 263, FP: 10760, FN: 186, PRECISION: 0.0239, RECALL: 0.5857, F1 SCORE: 0.0459
# Epoch 4: 7251 - TP: 236, FP: 7015, FN: 213, PRECISION: 0.0325, RECALL: 0.5256, F1 SCORE: 0.0613
# Epoch 5: 12770 - TP: 275, FP: 12495, FN: 174, PRECISION: 0.0215, RECALL: 0.6125, F1 SCORE: 0.0416

# Epoch 8: 3923 - TP: 187, FP: 3736, FN: 262, PRECISION: 0.0477, RECALL: 0.4165, F1 SCORE: 0.0855
